# Intro

In [1]:
import pandas as pd
pd.options.display.max_columns = 999

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn import linear_model


In [2]:
df = pd.read_csv("AmesHousing.tsv", delimiter = "\t")

In [3]:
# Create a function named transform_features() that, 
# for now, just returns the train data frame.

def transform_features(df):
    return df

In [4]:
# Create a function named select_features() that, 
# for now, just returns the Gr Liv Area and SalePrice 
# columns from the train data frame

def select_features(df):
    cols = ["Gr Liv Area", "SalePrice"]
    feature = df[cols]
    return feature

In [5]:
# Create a function named train_and_test()

def train_and_test(df):
    train = df[:1460]
    test = df[1460:]
    
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    num_train = train.select_dtypes(include = numerics)
    num_test = test.select_dtypes(include = numerics)
    
    features = num_train.columns.drop("SalePrice")
    
    lr = linear_model.LinearRegression()
    lr.fit(train[features], train["SalePrice"])
    predictions = lr.predict(test[features])
    
    mse = mean_squared_error(test["SalePrice"], predictions)
    rmse = mse**0.5
    
    return rmse

In [6]:
trans = transform_features(df)

select = select_features(trans)

train = train_and_test(select)

train

57088.25161263909

# Feature Engineering

Handle missing values
* All columns: Drop any with 5% or more missing values for now.
* Text columns: Drop any with 1 or more missing values for now.
* Numerical columns: For columns with missing values, fill in with the most common value in that column

In [7]:
df.shape

(2930, 82)

In [ ]:
missing = df[df.columns].isnull().sum()

In [ ]:
#drop columns with > 5% null values

over_5p = missing[(missing > df.shape[0]/20)]
df = df.drop(over_5p.index, axis = 1)
df

In [ ]:
#drop text coloumns with missing values

obj = ['object']
text_cols = df.select_dtypes(include = obj)
missing_text = text_cols.isnull().sum()
missing_text

In [ ]:
drop_text = missing_text[(missing_text > 0)]
drop_text

In [ ]:
df = df.drop(drop_text.index, axis = 1)
df

In [ ]:
#fill missing numerical values with the median

num_missing = df.isnull().sum()
nums_cols_missing = num_missing[(num_missing >0)]
nums_cols_missing

In [ ]:
df = df.fillna(df.median())

In [ ]:
df.isnull().sum()

In [ ]:
df.columns

* Create new features by comining other features
* Remove features that leak information about the sale.
* Remove features not useful for ML

In [ ]:
#are there houses sold before it was built?

years_sold = df["Yr Sold"] - df["Year Built"]
years_sold[(years_sold <0)]

In [ ]:
#are there houses remodled after it was sold?

years_since_remod = df["Yr Sold"] - df["Year Remod/Add"]
years_since_remod[(years_since_remod <0)]

In [ ]:
#creating better features and removing bad ones

df['Years Before Sale'] = years_sold
df['Years Since Remod'] = years_since_remod

df = df.drop(["Year Built", "Year Remod/Add"], axis = 1)

In [ ]:
#dropping rows with incorrect values

df = df.drop([1702, 2190, 2181], axis = 0)

In [ ]:
#drop coloumns not useful for ML

df = df.drop(["PID", "Order"], axis=1)

## Drop columns that leak info about the final sale
df = df.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)

In [ ]:
#update transform features 

def transform_features(df):
    num_missing = df.isnull().sum()
    drop_missing_cols = num_missing[(num_missing > len(df)/20)].sort_values()
    df = df.drop(drop_missing_cols.index, axis=1)
    
    text_mv_counts = df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)
    drop_missing_cols_2 = text_mv_counts[text_mv_counts > 0]
    df = df.drop(drop_missing_cols_2.index, axis=1)
    
    num_missing = df.select_dtypes(include=['int', 'float']).isnull().sum()
    fixable_numeric_cols = num_missing[(num_missing < len(df)/20) & (num_missing > 0)].sort_values()
    df = df.fillna(df.median())
    
    years_sold = df['Yr Sold'] - df['Year Built']
    years_since_remod = df['Yr Sold'] - df['Year Remod/Add']
    df['Years Before Sale'] = years_sold
    df['Years Since Remod'] = years_since_remod
    df = df.drop([1702, 2180, 2181], axis=0)

    df = df.drop(["PID", "Order", "Mo Sold", "Sale Condition", "Sale Type", "Year Built", "Year Remod/Add"], axis=1)
    return df


def select_features(df):
    return df[["Gr Liv Area", "SalePrice"]]


def train_and_test(df):
    train = df[:1460]
    test = df[1460:]
    
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    num_train = train.select_dtypes(include = numerics)
    num_test = test.select_dtypes(include = numerics)
    
    features = num_train.columns.drop("SalePrice")
    
    lr = linear_model.LinearRegression()
    lr.fit(train[features], train["SalePrice"])
    predictions = lr.predict(test[features])
    
    mse = mean_squared_error(test["SalePrice"], predictions)
    rmse = mse**0.5
    
    return rmse

df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)

rmse

# Feature Selection

In [ ]:
#heatmap showing correlation between features
%matplotlib inline

nums_only = df.select_dtypes(include = ["float64", "int64"])

plt.figure(figsize = (80,45))
sns.set(font_scale= 10)
feature_corr = sns.heatmap(nums_only.corr())


feature_corr


In [ ]:
abs_corr_coeffs = nums_only.corr()['SalePrice'].abs().sort_values(ascending = False)
abs_corr_coeffs

Features with strong correlation to Sale Price:
* Overall Qual
* Total Bsmt SF
* 1st Flr SF
* Gr Liv Area
* Garage Cars
* Garage Area

Features with decent correlation to Sale Price:
* Mas Vnr Area
* Full Bath
* Years Before Sale    
* Years Since Remod  

In [ ]:
#only keep columns where correlation is > 0.5

strong_features = abs_corr_coeffs[(abs_corr_coeffs > 0.5) & (abs_corr_coeffs < 1)]

In [ ]:
# Drop columns with less than 0.5 correlation with SalePrice

transform_df = transform_df.drop(abs_corr_coeffs[abs_corr_coeffs < 0.5].index, axis=1)


In [ ]:
## Create a list of column names from documentation that are *meant* to be categorical

nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]


Which categorical columns to keep?

Which columns are currently numerical but need to be encoded as categorical instead (because the numbers don't have any semantic meaning)?

If a categorical column has hundreds of unique values (or categories), should we keep it? When we dummy code this column, hundreds of columns will need to be added back to the data frame.